# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities_data_clean_df = pd.read_csv("city_data_clean.csv")
cities_data_clean_df=cities_data_clean_df.drop(["Unnamed: 0"], axis=1)
cities_data_clean_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,41.8500,-87.6500,79.43,67,75,1.99,US,1633821194
1,London,51.5085,-0.1257,55.15,91,53,2.30,GB,1633821217
2,Rockford,42.2711,-89.0940,79.00,57,75,9.22,US,1633821237
3,Sukkur,27.7052,68.8574,76.87,73,0,0.00,PK,1633821291
4,Sydney,-33.8679,151.2073,83.34,53,0,1.99,AU,1633821156
...,...,...,...,...,...,...,...,...,...
588,Ponta Delgada,37.7333,-25.6667,73.51,83,40,9.22,PT,1633822153
589,Srandakan,-7.9386,110.2506,62.73,87,50,3.09,ID,1633822154
590,Alexandria,31.2156,29.9553,73.36,83,0,8.05,EG,1633821970
591,Bucerías,20.7500,-105.3333,87.98,70,90,9.22,MX,1633822152


In [14]:

cities_data_clean_df.dropna(inplace = True) 
cities_data_clean_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,41.8500,-87.6500,79.43,67,75,1.99,US,1633821194
1,London,51.5085,-0.1257,55.15,91,53,2.30,GB,1633821217
2,Rockford,42.2711,-89.0940,79.00,57,75,9.22,US,1633821237
3,Sukkur,27.7052,68.8574,76.87,73,0,0.00,PK,1633821291
4,Sydney,-33.8679,151.2073,83.34,53,0,1.99,AU,1633821156
...,...,...,...,...,...,...,...,...,...
588,Ponta Delgada,37.7333,-25.6667,73.51,83,40,9.22,PT,1633822153
589,Srandakan,-7.9386,110.2506,62.73,87,50,3.09,ID,1633822154
590,Alexandria,31.2156,29.9553,73.36,83,0,8.05,EG,1633821970
591,Bucerías,20.7500,-105.3333,87.98,70,90,9.22,MX,1633822152


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
#configure gmaps
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


In [16]:
locations = cities_data_clean_df[["Lat", "Lng"]]

In [17]:
humidity = cities_data_clean_df["Humidity"].astype(float)
type(humidity)

pandas.core.series.Series

In [18]:
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=1,
                                 point_radius=1)

In [8]:
#Add layer to figure and display
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
#only_billy_and_peter = df.loc[(df["first_name"] == "Billy") | (df["first_name"] == "Peter"), :]


cities_data_clean_ideal_df = cities_data_clean_df.loc[ ((cities_data_clean_df["Max Temp"] > 70) & (cities_data_clean_df["Max Temp"] < 80)) & (cities_data_clean_df["Wind Speed"] < 10) &  (cities_data_clean_df["Cloudiness"] < 5) & (cities_data_clean_df["Humidity"] < 70), :    ]
cities_data_clean_ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
115,Saint George,37.1041,-113.5841,76.62,22,1,3.00,US,1633821480
134,North Platte,41.1239,-100.7654,77.38,29,1,4.00,US,1633821506
164,Abu Samrah,35.3029,37.1841,71.51,31,0,4.99,SY,1633821550
230,Laguna,38.4210,-121.4238,78.44,33,1,4.00,US,1633821419
279,Riyadh,24.6877,46.7219,77.14,17,1,3.51,SA,1633821522
336,Mahalapye,-23.1041,26.8142,74.28,21,0,7.96,BW,1633821796
337,Roseburg,43.2165,-123.3417,71.19,40,1,0.00,US,1633821597
354,Bagotville,-28.9833,153.4167,78.82,54,2,5.46,AU,1633821826
436,Ukiah,39.1502,-123.2078,77.58,35,1,8.05,US,1633821901
456,Ballina,-28.8667,153.5667,78.98,64,4,8.05,AU,1633821971


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = cities_data_clean_ideal_df


In [21]:
#  Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-21-037b27d67722>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,Saint George,37.1041,-113.5841,76.62,22,1,3.00,US,1633821480,
134,North Platte,41.1239,-100.7654,77.38,29,1,4.00,US,1633821506,
164,Abu Samrah,35.3029,37.1841,71.51,31,0,4.99,SY,1633821550,
230,Laguna,38.4210,-121.4238,78.44,33,1,4.00,US,1633821419,
279,Riyadh,24.6877,46.7219,77.14,17,1,3.51,SA,1633821522,
336,Mahalapye,-23.1041,26.8142,74.28,21,0,7.96,BW,1633821796,
337,Roseburg,43.2165,-123.3417,71.19,40,1,0.00,US,1633821597,
354,Bagotville,-28.9833,153.4167,78.82,54,2,5.46,AU,1633821826,
436,Ukiah,39.1502,-123.2078,77.58,35,1,8.05,US,1633821901,
456,Ballina,-28.8667,153.5667,78.98,64,4,8.05,AU,1633821971,


In [23]:
# Set parameters to search for hotels with 5000 meters.

type_word = "hotel"
radius = 5000

params = {
    "type": type_word,
    "radius": radius,
    "key": g_key
}

In [ ]:
#test
# params['location'] = "-34.8957, -60.0167"
# response = requests.get(base_url, params=params)
# print(response.url)

In [24]:
# Hit the Google Places API for each city's coordinates.

# base url

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    coords = f"{row['Lat']}, {row['Lng']}"
    params['location'] = coords
    print(f"Retrieving Results for city of: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result")
        


Retrieving Results for city of: Saint George.
Closest hotel is St. George.
Retrieving Results for city of: North Platte.


C:\Users\mpk99\Anaconda3\envs\PythonData38\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is North Platte.
Retrieving Results for city of: Abu Samrah.
Closest hotel is Suruj.
Retrieving Results for city of: Laguna.
Closest hotel is Sacramento.
Retrieving Results for city of: Riyadh.
Closest hotel is Riyadh.
Retrieving Results for city of: Mahalapye.
Closest hotel is Mahalapye.
Retrieving Results for city of: Roseburg.
Closest hotel is Roseburg.
Retrieving Results for city of: Bagotville.
Closest hotel is Wardell.
Retrieving Results for city of: Ukiah.
Closest hotel is Ukiah.
Retrieving Results for city of: Ballina.
Closest hotel is Ballina.
Retrieving Results for city of: Eureka.
Closest hotel is Eureka.
Retrieving Results for city of: Marshall.
Closest hotel is Guntersville.
Retrieving Results for city of: Abu Kamal.
Closest hotel is Al Bukamal.
Retrieving Results for city of: Al Bardīyah.
Closest hotel is Bardiyah.


In [25]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
115,Saint George,37.1041,-113.5841,76.62,22,1,3.00,US,1633821480,St. George
134,North Platte,41.1239,-100.7654,77.38,29,1,4.00,US,1633821506,North Platte
164,Abu Samrah,35.3029,37.1841,71.51,31,0,4.99,SY,1633821550,Suruj
230,Laguna,38.4210,-121.4238,78.44,33,1,4.00,US,1633821419,Sacramento
279,Riyadh,24.6877,46.7219,77.14,17,1,3.51,SA,1633821522,Riyadh
336,Mahalapye,-23.1041,26.8142,74.28,21,0,7.96,BW,1633821796,Mahalapye
337,Roseburg,43.2165,-123.3417,71.19,40,1,0.00,US,1633821597,Roseburg
354,Bagotville,-28.9833,153.4167,78.82,54,2,5.46,AU,1633821826,Wardell
436,Ukiah,39.1502,-123.2078,77.58,35,1,8.05,US,1633821901,Ukiah
456,Ballina,-28.8667,153.5667,78.98,64,4,8.05,AU,1633821971,Ballina


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
hotel_names = hotel_df["Hotel Name"].tolist()
cities = hotel_df["City"].tolist()
countries = hotel_df["Country"].tolist()

In [28]:
info_bx_hotel = [f"Hotel Name: {hotel}" for hotel in hotel_names]
info_bx_hotel

['Hotel Name: St. George',
 'Hotel Name: North Platte',
 'Hotel Name: Suruj',
 'Hotel Name: Sacramento',
 'Hotel Name: Riyadh',
 'Hotel Name: Mahalapye',
 'Hotel Name: Roseburg',
 'Hotel Name: Wardell',
 'Hotel Name: Ukiah',
 'Hotel Name: Ballina',
 'Hotel Name: Eureka',
 'Hotel Name: Guntersville',
 'Hotel Name: Al Bukamal',
 'Hotel Name: Bardiyah']

In [29]:
info_bx_city = [f"City: {city}" for city in cities]
info_bx_city

['City: Saint George',
 'City: North Platte',
 'City: Abu Samrah',
 'City: Laguna',
 'City: Riyadh',
 'City: Mahalapye',
 'City: Roseburg',
 'City: Bagotville',
 'City: Ukiah',
 'City: Ballina',
 'City: Eureka',
 'City: Marshall',
 'City: Abu Kamal',
 'City: Al Bardīyah']

In [30]:
info_bx_country = [f"Country: {country}" for country in countries]
info_bx_country

['Country: US',
 'Country: US',
 'Country: SY',
 'Country: US',
 'Country: SA',
 'Country: BW',
 'Country: US',
 'Country: AU',
 'Country: US',
 'Country: AU',
 'Country: US',
 'Country: US',
 'Country: SY',
 'Country: LY']

In [31]:
info_bx_list = []
for x in range(len(info_bx_city)):
    info_bx_list.append(f"{info_bx_hotel[x]} {info_bx_city[x]} {info_bx_country[x]}" )
                        
info_bx_list
    

['Hotel Name: St. George City: Saint George Country: US',
 'Hotel Name: North Platte City: North Platte Country: US',
 'Hotel Name: Suruj City: Abu Samrah Country: SY',
 'Hotel Name: Sacramento City: Laguna Country: US',
 'Hotel Name: Riyadh City: Riyadh Country: SA',
 'Hotel Name: Mahalapye City: Mahalapye Country: BW',
 'Hotel Name: Roseburg City: Roseburg Country: US',
 'Hotel Name: Wardell City: Bagotville Country: AU',
 'Hotel Name: Ukiah City: Ukiah Country: US',
 'Hotel Name: Ballina City: Ballina Country: AU',
 'Hotel Name: Eureka City: Eureka Country: US',
 'Hotel Name: Guntersville City: Marshall Country: US',
 'Hotel Name: Al Bukamal City: Abu Kamal Country: SY',
 'Hotel Name: Bardiyah City: Al Bardīyah Country: LY']

In [32]:
# Add marker layer ontop of heat map
locations_ideal = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations_ideal, info_box_content=info_bx_list)
#f"Hotel Name: {hotel}" for hotel in hotel_names,
# Add the layer to the map
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…